# Importing Libraries and Scripts
The code below consist of imports for libraries and scripts necessary for generating visualizations.

In [3]:
import sys
import random
import pandas as pd
import networkx as nx
import itables.options as opt
from auth import authenticate
from pyvis.network import Network
from itables import init_notebook_mode, show
from ontologies import get_classes, get_attributes, get_descriptions, read_individual

# Authenticating
The code below is responsible for the authentication of the provided credentials in order to access the model from the enpoint later.

In [4]:
status, message = authenticate(username='demo-user', password='demo')
if not status:
    sys.stdout.write(message)
else:
    sys.stdout.write('Provided credentials have been authenticated.')

Provided credentials have been authenticated.

# Acquiring the Classes, Attributes and Descriptions
The code below uses the token received from the step above to acquire models data from the available endpoints. **This step would fail if authentication was not completed successfuly prior to this step.**

In [5]:
if status:
    status, data = get_classes(access_token=message.get('access_token'))
    if status:
        df = pd.DataFrame(data=data)
    if data:
        status, descriptions = get_descriptions(access_token=message.get('access_token'))
        attributes = get_attributes(username='demo-user', password='demo', classes=data, cache=False)
else:
    sys.stdout.write('Aunthication is required before proceeding.\n')

Data for classes has been acquired successfuly.
Description for classes have been acquired successfuly.
Data for class attributes has been acquired successfuly.


In [ ]:
# config for displaying table better 
opt.lengthMenu = [2, 4, 8]
pd.set_option("display.max_rows", None)
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)
opt.maxBytes = 0
opt.classes = ['cell-border', "stripe", 'hover']
init_notebook_mode(all_interactive=True)


# cleaning for better more readable tables
df.columns = ['Class Label', 'Class IRI', 'Language', 'Leaf', 'Parent']
df = df[['Class Label', 'Language', 'Leaf', 'Parent', 'Class IRI']]
df['Leaf'] = df['Leaf'].fillna(False)
df['Language'] = df['Language'].fillna('')


# Generating DataFrame and other utility methods 

In [ ]:
classes = df.copy()

def get_classes_(labels, children=True, lang='en'):
    
    lang_classes = classes.copy()[classes['Language']==lang]
    resultant = pd.DataFrame()
    for label in labels:
        if children:
            chunk_df = lang_classes.copy()[(lang_classes['Class Label'] == label) | (lang_classes['Parent'] == label)]
        else:
            chunk_df = lang_classes.copy()[(lang_classes['Class Label'] == label)]
        resultant = pd.concat([resultant, chunk_df])
    return resultant

def get_leafs(lang='en'):
    
    leafs = classes.copy()[(classes['Leaf'] == True) & (classes['Language'] == lang)]
    return leafs

def get_non_leafs(lang='en'):
    
    non_leafs = classes.copy()[(classes['Leaf'] == False) & (classes['Language'] == lang)]
    return non_leafs

def get_children(lang='en'):
    
    children = classes.copy()[(classes['Parent'] != '') & (classes['Language'] == lang)]
    return children

def get_non_children(lang='en'):
    
    non_children = classes.copy()[(df['Parent'] == '') & (classes['Language'] == lang)]
    return non_children

def get_unqiue_classes(lang='en'):
    
    # unique_classes_df = classes.copy()[['Class Label', 'Language', 'Class IRI']]
    # unique_classes_df.drop_duplicates(subset='Class Label', inplace=True)
    # return unique_classes_df.copy()[unique_classes_df['Language'] == lang]
    unique_classes_df = classes.copy()[classes['Language'] == lang]
    unique_classes_df.drop_duplicates(subset='Class Label', inplace=True)
    return unique_classes_df[['Class Label', 'Leaf', 'Class IRI']]

def query_attrs(label):
    attrs = attributes.get(label, None)
    if attrs:
        attrs = pd.DataFrame(attrs)
    return attrs

def generate_graph(graph_df):
    
    G =  nx.from_pandas_edgelist(graph_df, source='Class Label', target='Parent', arrows=True)
    net = Network(notebook=True, height='500px', width='900px')
    net.from_nx(G, show_edge_weights=False)
    net.set_edge_smooth('dynamic')
    return net
# def generate_graph(graph_df):
    
    
# resultant_graph = generate_graph(graph_df=resultant_df)
# resultant_graph.show('resultant.html')

def generate_graph(graph_df):
    
    G =  nx.from_pandas_edgelist(graph_df, source='Class Label', target='Parent', arrows=True)
    net = Network(notebook=True, height='500px', width='900px')
    net.from_nx(G, show_edge_weights=False)
    net.set_edge_smooth('dynamic')
    return net
# def generate_graph(graph_df):
    
    
# resultant_graph = generate_graph(graph_df=resultant_df)
# resultant_graph.show('resultant.html')

def generate_visualization_(resultant_df, heading, legend):
    nodes = {}
    parents = {}
    for index, row in resultant_df.iterrows():
        class_label = row.pop('Class Label')
        if not class_label in nodes:
            nodes.update({class_label: {
                                        'n_id': random.randint(0,100),
                                        **row
                                        }
                        })
    #     parent = row.pop('Parent')
    #     if not parent in nodes:
    #         parents.update({parent: {
    #                                 'n_id': random.randint(0,100)
    #                                 }
    #                      })
    # for key, value in nodes.items():
    #     print(key, value)
    net = Network("700px", "900px", notebook=True, directed=True, heading=heading)
    for key, value in nodes.items():
        iri = df.copy()[df['Class Label']==key][['Class IRI']]
        iri.drop_duplicates(subset='Class IRI', inplace=True)
        iri = iri['Class IRI'].values[0]
        description = descriptions.get(iri, None)
        if description:
            description = description.get('en')
        else:
            description = 'Description not available'
        if value.get('Leaf'):
            color = '#AED6F1'
        else:
            color = '#ABEBC6'
        net.add_node(n_id=value.get('n_id'), label=key, title=description, color=color, shape='dot')
    if legend:
        leg1= net.add_node(n_id=-1, label='Leafs', title='Representation of leaf nodes', color='#AED6F1', shape='dot', x=-400, y=-500, physics=False)
        leg2 = net.add_node(n_id=-2, label='Non-Leafs', title='Representation of non-leaf nodes', color='#ABEBC6', shape='dot', x=-400, y=-400, physics=False)

    
    
    for key, value in nodes.items():
        parent = value.get('Parent', None)
    #     print(parent)
        if parent:
            _to = value.get('n_id', None)
            _from = nodes.get(parent, {}).get('n_id', None)
            if _to and _from:
                net.add_edge(_to, _from, arrowStrikethrough=True)
    return net

def generate_table(df, num, title):
    show(df,
    columnDefs=[
        {   "width": "120px",
             "className": "dt-center",
            "targets": "_all",
            "createdCell": """
            function (td, cellData, rowData, row, col) {
                console.log(rowData)
                $(td).css('word-wrap', 'break-word')
                if (cellData == false) {
                    $(td).css('color', '#FE0707')
                    $(td).css('font-weight', 'bold')
                }
                else if (cellData == true) {
                    $(td).css('color', '#1892E7')
                    $(td).css('font-weight', 'bold')
                }
            }
            """,
        }
    ],
    eval_functions=True,
    tags=f'<caption style="caption-side:bottom;padding:10px;"><i>Table {num}: </i>{title}</caption>')

# Individuals Data

In [ ]:
status, message = authenticate(username='demo-user', password='demo')
if not status:
    sys.stdout.write(message)
else:
    sys.stdout.write('Provided credentials have been authenticated.')
    
status, individuals = read_individual(access_token=message.get('access_token'))
individuals_df = pd.DataFrame(data=individuals)
individuals_df.columns = ['IRI', 'Family Name', 'Given Name']
individuals_df = individuals_df[['Family Name', 'Given Name', 'IRI']]
generate_table(individuals_df, 1, 'Individuals data')

In [ ]:
en_unique_classes = get_unqiue_classes(lang='en')
generate_table(en_unique_classes, 2, 'Classes available in English')

In [ ]:
de_unique_classes = get_unqiue_classes(lang='de')
generate_table(de_unique_classes, 3, 'Classes available in German')

In [ ]:
fr_unique_classes = get_unqiue_classes(lang='fr')
generate_table(fr_unique_classes, 4, 'Classes available in French')

In [ ]:
en_leafs = get_leafs(lang='en')
generate_table(fr_unique_classes, 5, 'Leaf classes available in English')

In [ ]:
visual = generate_visualization_(en_leafs, 'Leaf Classes Available in English', False)
visual.show('en_leafs.html')

In [ ]:
fr_leafs = get_leafs(lang='fr')
generate_table(fr_leafs, 6, 'Leaf classes available in French')

In [ ]:
visual = generate_visualization_(fr_leafs, 'Leaf Classes Available in French', False)
visual.show('fr_leafs.html')

In [ ]:
de_leafs = get_leafs(lang='de')
generate_table(de_leafs, 7, 'Leaf classes available in German')

In [ ]:
en_non_leafs = get_non_leafs(lang='en')
generate_table(en_non_leafs, 8, 'Non-leaf classes available in English')

In [ ]:
de_non_leafs = get_non_leafs(lang='de')
generate_table(de_non_leafs, 9, 'Non-leaf classes available in German')

In [ ]:
visual = generate_visualization_(de_non_leafs, 'Non-leaf classes available in German', False)
visual.show('fr_leafs.html')

In [ ]:
fr_non_leafs = get_non_leafs(lang='fr')
generate_table(fr_non_leafs, 10, 'Non-leaf classes available in French')

In [ ]:
en_children = get_children(lang='en')
generate_table(en_children, 11, 'Children classes available in English')

In [ ]:
visual = generate_visualization_(en_children, 'Children classes available in English', False)
visual.show('en_children.html')

In [ ]:
fr_children = get_children(lang='fr')
generate_table(fr_children, 12, 'Children classes available in French')

In [ ]:
de_children = get_children(lang='de')
generate_table(fr_children, 13, 'Children classes available in German')

In [ ]:
en_non_children = get_non_children(lang='en')
generate_table(en_non_children, 14, 'Non-children classes available in English')

In [ ]:
fr_non_children = get_non_children(lang='fr')
generate_table(en_non_children, 15, 'Non-children classes available in French')

In [ ]:
visual = generate_visualization_(fr_non_children, 'Non-children classes available in French', False)
visual.show('fr_non_children.html')

In [ ]:
de_non_children = get_non_children(lang='de')
generate_table(de_non_children, 16, 'Non-children classes available in German')

In [ ]:
labels = ['Person']
resultant_df = get_classes_(labels=labels, children=True, lang='en')
generate_table(resultant_df, 17, 'Person Class')

In [ ]:
net = generate_visualization_(resultant_df, 'Person Class', False)
net.show('example.html')

In [ ]:
labels = ['Business Object', 'Bibliographical object', 'Editorial Object', 'Editorial object']
resultant_df = get_classes_(labels=labels, children=True, lang='en')
generate_table(resultant_df, 18, 'Business Object and other classes')

In [ ]:
def generate_visualization_(resultant_df, heading, legend):
    nodes = {}
    parents = {}
    for index, row in resultant_df.iterrows():
        class_label = row.pop('Class Label')
        if not class_label in nodes:
            nodes.update({class_label: {
                                        'n_id': random.randint(0,100),
                                        **row
                                        }
                        })
    #     parent = row.pop('Parent')
    #     if not parent in nodes:
    #         parents.update({parent: {
    #                                 'n_id': random.randint(0,100)
    #                                 }
    #                      })
    # for key, value in nodes.items():
    #     print(key, value)

    net = Network("700px", "900px", notebook=True, directed=True, heading=heading)
    for key, value in nodes.items():
        iri = df.copy()[df['Class Label']==key][['Class IRI']]
        iri.drop_duplicates(subset='Class IRI', inplace=True)
        iri = iri['Class IRI'].values[0]
        description = descriptions.get(iri, None)
        if description:
            description = description.get('en')
        else:
            description = 'Description not available'
        if value.get('Leaf'):
            color = '#AED6F1'
        else:
            color = '#ABEBC6'
        net.add_node(n_id=value.get('n_id'), label=key, title=description, color=color, shape='dot', size=20)
    if legend:
        leg1= net.add_node(n_id=-1, group=99, label='Leafs', title='Representation of leaf nodes', color='#AED6F1', shape='box', x=360, y=-500, physics=False, fixed=True, size=10, widthConstraint= 5)
        leg2 = net.add_node(n_id=-2, group=99, label='Non-Leafs', title='Representation of non-leaf nodes', color='#ABEBC6', shape='box', x=360, y=-430, physics=False, fixed=True, size=10, widthConstraint= 5)

    
    
    for key, value in nodes.items():
        parent = value.get('Parent', None)
    #     print(parent)
        if parent:
            _to = value.get('n_id', None)
            _from = nodes.get(parent, {}).get('n_id', None)
            if _to and _from:
                net.add_edge(_to, _from, arrowStrikethrough=True)
    return net
net = generate_visualization_(resultant_df, 'Business Object and its Children Classes', True)
net.show('example.html')

In [ ]:
labels = ['Picture']
resultant_df = get_classes_(labels=labels, children=True, lang='en')
generate_table(resultant_df, 19, 'Picture class')

In [ ]:
net = generate_visualization_(resultant_df, 'Picture and its Children Classes', True)
net.show('example.html')

In [15]:
def query_attrs(label):
    attrs = attributes.get(label, None)
    if attrs:
        attrs = pd.DataFrame(attrs)
    return attrs

def generate_table(df, num, title):
    show(df,
    columnDefs=[
        {   "width": "120px",
             "className": "dt-center",
            "targets": "_all",
            "createdCell": """
            function (td, cellData, rowData, row, col) {
                console.log(rowData)
                $(td).css('word-wrap', 'break-word')
                if (cellData == false) {
                    $(td).css('color', '#FE0707')
                    $(td).css('font-weight', 'bold')
                }
                else if (cellData == true) {
                    $(td).css('color', '#1892E7')
                    $(td).css('font-weight', 'bold')
                }
            }
            """,
        }
    ],
    eval_functions=True,
    tags=f'<caption style="caption-side:bottom;padding:10px;"><i>Table {num}: </i>{title}</caption>')

In [25]:
classes = ['http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#Season', 'http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#Document', 'http://www.ebu.ch/metadata/ontologies/ebucore/ebucore#Track']

In [32]:
attrs = query_attrs(classes[2])
if not isinstance(attrs, list):
    generate_table(attrs, 19, 'Available attributes')
else:
    print('Attributes not found')